In [73]:

import pandas as pd  
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request 
from PIL import Image 

from sklearn.feature_extraction.text import CountVectorizer

In [46]:
df_main = pd.read_csv('anime-dataset-2023.csv')

In [74]:
df = pd.read_csv('anime-dataset-2023.csv')[['anime_id', 'Name', 'Score', 'Genres', 'Type', 'Episodes', 'Synopsis', 'Image URL']]
df

,anime_id,Name,Score,Genres,Type,Episodes,Synopsis,Image URL
0,1,Cowboy Bebop,8.75,"Action, Award Winning, Sci-Fi",TV,26.0,"Crime is timeless. By the year 2071, humanity ...",https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,8.38,"Action, Sci-Fi",Movie,1.0,"Another day, another bounty—such is the life o...",https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,8.22,"Action, Adventure, Sci-Fi",TV,26.0,"Vash the Stampede is the man with a $$60,000,0...",https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,7.25,"Action, Drama, Mystery, Supernatural",TV,26.0,Robin Sena is a powerful craft user drafted in...,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,6.94,"Adventure, Fantasy, Supernatural",TV,52.0,It is the dark century and the people are suff...,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...
24900,55731,Wu Nao Monu,UNKNOWN,"Comedy, Fantasy, Slice of Life",ONA,15.0,No description available for this anime.,https://cdn.myanimelist.net/images/anime/1386/...
24901,55732,Bu Xing Si: Yuan Qi,UNKNOWN,"Action, Adventure, Fantasy",ONA,18.0,No description available for this anime.,https://cdn.myanimelist.net/images/anime/1383/...
24902,55733,Di Yi Xulie,UNKNOWN,"Action, Adventure, Fantasy, Sci-Fi",ONA,16.0,No description available for this anime.,https://cdn.myanimelist.net/images/anime/1130/...
24903,55734,Bokura no Saishuu Sensou,UNKNOWN,UNKNOWN,Music,1.0,A music video for the song Bokura no Saishuu S...,https://cdn.myanimelist.net/images/anime/1931/...


In [67]:
list_columns = [x for x in df.columns]
list_unique = []
for x in df.columns:
    y = df[x].unique()
    list_unique.append(y)

df_unique = pd.DataFrame(list_columns)
df_unique['unqiue']=list_unique
df_unique

,0,unqiue
0,anime_id,"[1, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 21, 22..."
1,Name,"[Cowboy Bebop, Cowboy Bebop: Tengoku no Tobira..."
2,Score,"[8.75, 8.38, 8.22, 7.25, 6.94, 7.92, 8.0, 7.55..."
3,Genres,"[Action, Award Winning, Sci-Fi, Action, Sci-Fi..."
4,Type,"[TV, Movie, OVA, Special, ONA, Music, UNKNOWN]"
5,Episodes,"[26.0, 1.0, 52.0, 145.0, 24.0, 74.0, 220.0, UN..."
6,Synopsis,"[Crime is timeless. By the year 2071, humanity..."
7,Image URL,[https://cdn.myanimelist.net/images/anime/4/19...
8,Studios,"[Sunrise, Bones, Madhouse, Toei Animation, Gal..."


Dropping anime with unknow scores

In [76]:
for x in df:
    df[df[x]=='UNKNOWN']
    df[x].replace('UNKNOWN', value = np.nan, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_15668\883659413.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[x].replace('UNKNOWN', value = np.nan, inplace=True)


In [77]:
df.isna().sum()

anime_id        0
Name            0
Score        9213
Genres       4929
Type           74
Episodes      611
Synopsis        0
Image URL       0
dtype: int64

Dropping all missing values

In [78]:
df.dropna( inplace=True)

In [79]:
df.shape

(13852, 8)

In [80]:
df.columns

Index(['anime_id', 'Name', 'Score', 'Genres', 'Type', 'Episodes', 'Synopsis',
       'Image URL'],
      dtype='object')

In [81]:
df.to_csv('anime-final.csv')

# Recc

In [55]:
df['Genres'] = df['Genres'].fillna('')  # Fill NaN values with empty string


# Create a CountVectorizer object
cv = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Fit and transform the combined genre and type column
genre_matrix = cv.fit_transform(df['Genres'])

# Get the feature names (genres and types)
genre_names = cv.get_feature_names_out()

# Create a DataFrame with genre and type counts, using anime_id as the index
genre_df = pd.DataFrame(genre_matrix.toarray(), columns=genre_names, index=df['anime_id']).reset_index()

genre_df



# Create a CountVectorizer object
cv = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Fit and transform the combined genre and type column
type_matrix = cv.fit_transform(df['Type'])

# Get the feature names (genres and types)
type_names = cv.get_feature_names_out()

# Create a DataFrame with genre and type counts, using anime_id as the index
type_df = pd.DataFrame(type_matrix.toarray(), columns=type_names, index=df['anime_id']).reset_index()

type_df

c:\Users\User\Documents\anime-recommendation-app\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\User\Documents\anime-recommendation-app\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,anime_id,movie,music,ona,ova,special,tv
0,1,0,0,0,0,0,1
1,5,1,0,0,0,0,0
2,6,0,0,0,0,0,1
3,7,0,0,0,0,0,1
4,8,0,0,0,0,0,1
...,...,...,...,...,...,...,...
11478,55247,0,1,0,0,0,0
11479,55288,0,1,0,0,0,0
11480,55289,0,0,0,0,1,0
11481,55339,0,0,0,0,1,0


In [56]:


# Create a CountVectorizer object
cv = CountVectorizer(tokenizer= lambda x: x.split(', '))

# Fit and transform the combined genre and type column
type_matrix = cv.fit_transform(df['Type'])

# Get the feature names (genres and types)
type_names = cv.get_feature_names_out()

# Create a DataFrame with genre and type counts, using anime_id as the index
type_df = pd.DataFrame(type_matrix.toarray(), columns=type_names, index=df['anime_id']).reset_index()

type_df

c:\Users\User\Documents\anime-recommendation-app\venv\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,anime_id,movie,music,ona,ova,special,tv
0,1,0,0,0,0,0,1
1,5,1,0,0,0,0,0
2,6,0,0,0,0,0,1
3,7,0,0,0,0,0,1
4,8,0,0,0,0,0,1
...,...,...,...,...,...,...,...
11478,55247,0,1,0,0,0,0
11479,55288,0,1,0,0,0,0
11480,55289,0,0,0,0,1,0
11481,55339,0,0,0,0,1,0


,anime_id,Name,Score,Genres,Type,Episodes,Synopsis,Image URL,Studios


## Modelling

In [57]:
genre_type_cosine = genre_df.merge(type_df).drop(columns='anime_id')
genre_type_df = genre_df.merge(type_df)
genre_type_cosine_matrix = cosine_similarity(genre_type_cosine)  

# Target Value
target = df[df['Name']=='Cowboy Bebop']
target_id =int(target[['anime_id']].values)

df_target = genre_type_df[genre_type_df['anime_id']==target_id]
df_target

# Target cosine with others
cosine_sim = cosine_similarity(df_target.drop(columns='anime_id'), genre_type_cosine)
cosine_sim_df = pd.DataFrame(cosine_sim).transpose().sort_values(0, ascending=False).rename(columns = {0:'Compatibility Score'})
cosine_sim_df['Compatibility Score'] = cosine_sim_df['Compatibility Score'].apply(lambda x: f'{round(x*100,2)}%' if x < 1 else f'{x*100}%')

#Getting anime id based off index
list_recommended_index = cosine_sim_df.index.tolist() # Exclude first one cos its the anime itself

recc_list = genre_type_df.iloc[list_recommended_index]['anime_id'].values.tolist()
cosine_sim_df['anime_id'] = recc_list


df_reccomended = df.iloc[list_recommended_index][df['anime_id']!=target_id]
df_final = df_reccomended.merge(cosine_sim_df)

df_final.head(20)

C:\Users\User\AppData\Local\Temp\ipykernel_15668\84923878.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  target_id =int(target[['anime_id']].values)
C:\Users\User\AppData\Local\Temp\ipykernel_15668\84923878.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_reccomended = df.iloc[list_recommended_index][df['anime_id']!=target_id]


,anime_id,Name,Score,Genres,Type,Episodes,Synopsis,Image URL,Studios,Compatibility Score
0,467,Koukaku Kidoutai: Stand Alone Complex,8.42,"Action, Award Winning, Sci-Fi",TV,26.0,"In the not so distant future, mankind has adva...",https://cdn.myanimelist.net/images/anime/11/50...,Production I.G,100.0%
1,35847,SSSS.Gridman,7.12,"Action, Award Winning, Sci-Fi",TV,12.0,Yuuta Hibiki wakes up in the room of Rikka Tak...,https://cdn.myanimelist.net/images/anime/1973/...,Trigger,100.0%
2,43229,Godzilla: S.P,6.65,"Action, Award Winning, Mystery, Sci-Fi",TV,13.0,"This series features an original story, which ...",https://cdn.myanimelist.net/images/anime/1447/...,"Bones, Orange",89.44%
3,3572,Macross F,7.89,"Action, Award Winning, Romance, Sci-Fi",TV,25.0,Following a catastrophic war against a race of...,https://cdn.myanimelist.net/images/anime/9/156...,Satelight,89.44%
4,1575,Code Geass: Hangyaku no Lelouch,8.7,"Action, Award Winning, Drama, Sci-Fi",TV,25.0,"In the year 2010, the Holy Empire of Britannia...",https://cdn.myanimelist.net/images/anime/1032/...,Sunrise,89.44%
5,2001,Tengen Toppa Gurren Lagann,8.63,"Action, Adventure, Award Winning, Sci-Fi",TV,27.0,Simon and Kamina were born and raised in a dee...,https://cdn.myanimelist.net/images/anime/4/512...,Gainax,89.44%
6,2904,Code Geass: Hangyaku no Lelouch R2,8.91,"Action, Award Winning, Drama, Sci-Fi",TV,25.0,"One year has passed since the Black Rebellion,...",https://cdn.myanimelist.net/images/anime/1088/...,Sunrise,89.44%
7,2521,Haja Kyosei G Dangaiou,5.37,"Action, Sci-Fi",TV,13.0,A mysterious explosion on Futagami Island begi...,https://cdn.myanimelist.net/images/anime/1303/...,AIC,86.6%
8,5883,Examurai Sengoku,6.18,"Action, Sci-Fi",TV,24.0,The Examurai Sengoku anime production depicts ...,https://cdn.myanimelist.net/images/anime/13/33...,TMS Entertainment,86.6%
9,36828,Caligula (TV),6.0,"Action, Sci-Fi",TV,12.0,"What is happiness? Ever the fan of psychology,...",https://cdn.myanimelist.net/images/anime/1910/...,Satelight,86.6%
